In [1]:
import numpy as np
import open3d as o3d
from read_datas import ReadDatas

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas('..\Point Cloud Data\Corner.ply')
pcd.gpf_ground_extraction()

In [3]:
o3d.visualization.draw_geometries([pcd.datas])

In [4]:
ground = pcd.np_to_o3d(pcd.ground)

In [5]:
o3d.visualization.draw_geometries_with_editing([ground])


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [6]:
def paves_cutting(ground):    
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name='Open3D', visible=True)
    vis.add_geometry(ground)
    vis.run()
    geometry = vis.get_cropped_geometry()
    vis.destroy_window()
    geometry = np.asarray(geometry.points)

    return geometry

In [7]:
def seed_select(ground):    
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name='Open3D', visible=True)
    vis.add_geometry(ground)
    vis.run()
    seed = vis.get_picked_points()
    vis.destroy_window()


    return seed

In [8]:
def find_index(list1,list2):
    index = []
    for i in list2:
        a = np.argwhere((list1 == i).all(axis=1)).ravel()
        if a:
            index.append(a[0])
    return index

In [9]:
def no_paves(ground):
    a = True
    geometrys = np.zeros((1,3))

    while a:
        geometry = paves_cutting(ground)
        geometrys = np.concatenate((geometrys,geometry),axis=0)
        input_1 = input('按Y或者y继续')
        if input_1 == 'y' or input_1 == 'Y':
            a = True
        else:
            a = False
    return geometrys

In [10]:
ground1 = o3d.io.read_point_cloud('cropped_1.ply')
ground2 = o3d.io.read_point_cloud('cropped_2.ply')
ground3 = o3d.io.read_point_cloud('cropped_3.ply')


In [11]:
ground_1 = np.asarray(ground1.points)
ground_2 = np.asarray(ground2.points)
ground_3 = np.asarray(ground3.points)

In [12]:
first = pcd.ground

In [13]:

seed1= []
for i in ground_1:
    seed1 += np.where((first == i).all(axis = 1))[0].tolist()

In [14]:
for i in ground_2:
    seed1 += np.where((first == i).all(axis = 1))[0].tolist()

In [15]:
for i in ground_3:
    seed1 += np.where((first == i).all(axis = 1))[0].tolist()


In [16]:
no_pavas = np.array(seed1)

In [17]:
curvity = pcd.curvature_calculation(ground)
print('-----------')
seed = np.argmin(curvity)
seed = np.array([seed])
print('-----------')
neighbors = pcd.find_neighbour_points(ground)
print('-----------')


-----------
-----------
-----------


In [18]:
seed = seed_select(ground)

In [19]:
paves1 = no_paves(ground)


In [20]:

no_paves1 = np.array(find_index(pcd.ground,paves1))


C:\Users\master\AppData\Local\Temp\ipykernel_21560\3836748341.py:5: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if a:


In [17]:
paves = pcd.region_growing_segmentation(seed, clouds = ground,no_paves = no_paves1)

In [ ]:
inliers = list(paves)
inlier_cloud = ground.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = ground.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([0,1,0])
o3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])